In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt
import Fvvsc

# 1. Data matrix
- (1a) create p array and data matrix
- (1b) load data from delaney's folder

Each energy value corresponds to a different density matrix rho. This means that each energy value corresponds to a list of 4 numbers to create density matrix rho, which is that p0xyz vector (P0,P). 

So the point of the p array is to create a list of energy values.

We also have a matrix of data, data_matrix, which is an Nx4 matrix with N rows and 4 columns (P0, Px, Py, Pz). so each row of data_matrix is a (P0, P) vector. To choose rows (To index rows) we will use the p array. So like p[5] would be an energy value in the p array , and data_matrix[5,:] would be a (P0,P) vector for energy value number 5. 

I think data matrix comes from choosing a time step of delaneys code (the nu3D = rawdata['nu3D'] stuff)

In [2]:
#1a 
N = 10
B = 2
emax = 10
p = np.linspace(0, emax, N) #these are the energy values # this is the p1 in the F function?
deltap = p[1]-p[0]
emax = (N-1)/deltap
Nbig = 1 + B*(N-1)
ebig = (Nbig-1)*deltap


zero_array = np.zeros(4) # for out of range energy values, but as global variable 
Nbig = B*(N-1)

print(p)

#data_matrix = np.zeros((N, 4)) #should be N x 4
#p1 = data_matrix[:, 0] # this means first column
#p2 = data_matrix[:, 1] # second column
#p3 = data_matrix[:, 2] # third column
#F(p1, p2, p3) = data_matrix[:, 3] #last column is 

#wait im thinking about an N by 4 matrix...
# what i need to be thinking about is a 4d array

#data_matrix[p1, p2, p3, F(p1, p2, p3)]
# p1 chooses the set of matrices? (so its like the energy value bc each energy has a different matrix)
    # first index is p1/deltap
# p2 chooses the actual matrix in that set??
    
# p3 chooses the term in the matrix (which is a 4 element array) that is F(p1, p2, p3)?
# last index would pick a value from the 4 values that F produces?

#QUESTION: so is p array p1?



[ 0.          1.11111111  2.22222222  3.33333333  4.44444444  5.55555556
  6.66666667  7.77777778  8.88888889 10.        ]


In [3]:
#1b
data = np.load('Run35.npz', allow_pickle = True)
rawdata = data['raw'].item() #loads dictionary
rawdata.keys() # shows stored info

dict_keys(['initial array', 'time', 'dt', 'dN', 'nu3D', 'nubar3D', 'nu_init matrix', 'nu_bar_init matrix'])

In [4]:
nu3D = rawdata['nu3D'] #3D matrix N_time x N x 4
# N_time is the number of time steps so [0,:,:] is a matrix for the first time steps
nu3D
nu3D[1,:,:] # is an Nx4 matrix for timestep 1

array([[ 4.87502836e-01, -4.27087541e-02, -2.79853529e-02,
         9.98695088e-01],
       [ 4.75021265e-01, -3.79413304e-02, -1.43663969e-02,
         9.99175790e-01],
       [ 4.62570815e-01, -3.70438631e-02, -9.60371930e-03,
         9.99266153e-01],
       [ 4.50166860e-01, -3.67286724e-02, -7.19154234e-03,
         9.99297633e-01],
       [ 4.37824541e-01, -3.65825995e-02, -5.73316134e-03,
         9.99312008e-01],
       [ 4.25558699e-01, -3.65032048e-02, -4.75448149e-03,
         9.99319644e-01],
       [ 4.13383799e-01, -3.64553175e-02, -4.05074076e-03,
         9.99324102e-01],
       [ 4.01313869e-01, -3.64242310e-02, -3.51913840e-03,
         9.99326869e-01],
       [ 3.89362435e-01, -3.64029153e-02, -3.10242300e-03,
         9.99328657e-01],
       [ 3.77542466e-01, -3.63876660e-02, -2.76618918e-03,
         9.99329842e-01],
       [ 3.65866325e-01, -3.63763811e-02, -2.48851873e-03,
         9.99330635e-01],
       [ 3.54345718e-01, -3.63677955e-02, -2.25478808e-03,
      

In [5]:
nu3D[0,0,:]

array([0.4875026, 0.       , 0.       , 1.       ])

# 2 Create function
- (2a) creat function:
    - input N x 4 matrix with all the density matrix info
    - output: N x N$_{\mathrm{big}}$ x N$_{\mathrm{big}}$ x 4 matrix that holds the components of F(p1, p2, p3)

compute_F_matrix function: "store_all_Fvvsc_combos"

Starts with index i  when i is 0 it goes to density matrix, looks at index 0 (which is the energy value and then it assigns the 4 values that correspond to that energy value (which is the colon here) assigns it to the vector p0xyz_1….. then we go to index k, index k starts at 0 now, and it goes into density matrix, picks out index 0 and assigns the 4 values corresponding the that energy value (the energy value that index 0 meant) and assigns it to the vector p0xyz_2, same thing for vector p0xyz_3 and index k, and then finally it takes these 3 energy values (but in terms of index of density matrix I j k) and calculates what index energy 4 should have (like based on e1 e2 e3 what should 34 be? Well whatever it is its gonna have index l in density matrix, which is I + j -k. Then it uses that calculated index l, to find the correct p0xyz_4 vector associated with that energy value.

Now we have 4 (P0, P) vectors, remember they all correspond to a certain rho matrix in Fvvsc, so we take the function Fvvsc_components, and we input these chosen p0xyz vectors into that function. And we store this outcome in “F_matrix”.

This is what happens after one run through of the for loop. After every run-through of the for loop, we now have every combination of inputs into Fvvsc_components  saved into one spot and that spot is the F_matrix which we can now index to get which combo we want. (Every combo on inputs info Fvvsc_components means every possible combo of rho matrices meaning every possible combo of energies)

Now we want to use delaneys nu3d data as the density matrix input. 

Eventually the integral is a function of p1, I(p1) 

In [6]:
#@nb.jit(nopython=True)
def store_all_Fvvsc_combos(density_matrix):
    # Initialize the output matrix with zeros
    F_matrix = np.zeros((N, Nbig, Nbig, 4), dtype=np.complex128)

    for i in range(N): # dimension i is N because p1 goes from 0 to emax
        for j in range(Nbig): # dimension j is Nbig because p2 goes from 0 to B*emax
            for k in range(Nbig): # dimension k is Nbig because p3 goes from 0 to B*emax
                
                l = i + j - k # because p4 = p1 + p2 - p3 (feynmann diagram)
    
                p0xyz_1 = density_matrix[i, :]
            
                if j < N:
                    p0xyz_2 = density_matrix[j, :]
                else: 
                    zero_array
                
                if k < N:
                    p0xyz_3 = density_matrix[k, :]
                else: 
                    zero_array
                
                if l < 0 or l >= N:
                # when i didnt do this and just had l<0 i had problems... so here ya go
                    p0xyz_4 = zero_array 
                else:
                    p0xyz_4 = density_matrix[l, :]
               
            # WAIT THIS IS BACKWARDS
            
                # get F components for this combination of energy values (is it energies?)
                F_matrix[i, j, k, :] = Fvvsc.Fvvsc_components(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)

    return F_matrix


In [7]:
#print(nu3D)

In [8]:
F_matrix = store_all_Fvvsc_combos(nu3D[0,:,:]) # bc this is Nx4

F_matrix

array([[[[0.24968773+0.j, 0.        +0.j, 0.        +0.j,
          0.24968773+0.j],
         [0.24953184+0.j, 0.        +0.j, 0.        +0.j,
          0.24953184+0.j],
         [0.25544985+0.j, 0.        +0.j, 0.        +0.j,
          0.25544985+0.j],
         ...,
         [0.29586586+0.j, 0.        +0.j, 0.        +0.j,
          0.29586586+0.j],
         [0.29586586+0.j, 0.        +0.j, 0.        +0.j,
          0.29586586+0.j],
         [0.29586586+0.j, 0.        +0.j, 0.        +0.j,
          0.29586586+0.j]],

        [[0.24922025+0.j, 0.        +0.j, 0.        +0.j,
          0.24922025+0.j],
         [0.24922025+0.j, 0.        +0.j, 0.        +0.j,
          0.24922025+0.j],
         [0.24890943+0.j, 0.        +0.j, 0.        +0.j,
          0.24890943+0.j],
         ...,
         [0.28829065+0.j, 0.        +0.j, 0.        +0.j,
          0.28829065+0.j],
         [0.28829065+0.j, 0.        +0.j, 0.        +0.j,
          0.28829065+0.j],
         [0.28829065+0.j, 0.       

# 3. Complication

In [9]:
def store_all_Fvvsc_combos_shorter(density_matrix):
    # Initialize the output matrix with zeros
    F_matrix = np.zeros((N, Nbig, Nbig, 4), dtype=np.complex128)

    for i in range(N): # dimension i is N because p1 goes from 0 to emax
        for j in range(Nbig): # dimension j is Nbig because p2 goes from 0 to B*emax
            for k in range(Nbig): # dimension k is Nbig because p3 goes from 0 to B*emax
                
                l = i + j - k # because p4 = p1 + p2 - p3 (feynmann diagram)
            
                if i and j and k and l < N:
                    F_matrix[i, j, k, :] = Fvvsc.Fvvsc_components(density_matrix[(i-1), :], density_matrix[(j-1), :], density_matrix[(k-1), :], p0xyz_4 = density_matrix[(l-1), :])
                else:
                    zero_array

    return F_matrix

In [10]:
F_matrix = store_all_Fvvsc_combos_shorter(nu3D[0,:,:]) # bc this is Nx4

F_matrix

array([[[[0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         ...,
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j]],

        [[0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         ...,
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j],
         [0.        +0.j, 0.       

QUESTIONS:
1. Not sure what I am supposed to do with the delta stuff?
2. also should I have different arrays that go to eps_big?
3. the "Shorter" function was trying to implement the last page of QKE 4 but idk if im understanding correctly